In [1]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from obspy.io.quakeml.core import Unpickler
import pandas as pd
from libcomcat.dataframes import get_phase_dataframe, get_magnitude_data_frame
from libcomcat.search import get_event_by_id
import os

client = Client("IRIS")

mseed_savedir = '/data/sdybing/olderdata/mseeds/'
pre_post_sec = 60

In [2]:
# CSV catalog from USGS website search tool
new_catalog = np.genfromtxt('/home/sdybing/neic-mlaapde/2000-2013_catalog.csv', delimiter = ',', dtype = 'str', skip_header = 1) # M7.5+ 01/01/2000 - 07/31/2013

In [9]:
new_catalog[:,8]

array(['usp0009qb4', 'usp0009sbh', 'usp0009txv', 'usp0009ut0',
       'usp000a3qq', 'usp000a3sp', 'usp000a45f', 'usp000a70h',
       'usp000a7m5', 'usp000a8ds', 'official20010623203314130_33',
       'usp000aj40', 'usp000ar78', 'usp000asvm', 'usp000azqw',
       'usp000ba6g', 'usp000ba6m', 'usp000bbpf', 'usp000be54',
       'ak002e435qpj', 'usp000bnyr', 'usp000c2ka', 'usp000c41f',
       'official20030925195006360_27', 'usp000cd1n', 'usp000d85g',
       'usp000db93', 'official20041226005853450_30',
       'official20050328160936530_30', 'usp000dsw1', 'usp000dz2j',
       'usp000e09h', 'usp000e12e', 'usp000e8ys', 'usp000ef1h',
       'usp000eg5g', 'usp000ensm', 'usp000exfn', 'usp000f2ab',
       'usp000f34b', 'usp000f83m', 'usp000fj6k', 'usp000fjta',
       'official20070912111026830_34', 'usp000fn2b', 'usp000fpca',
       'usp000fshy', 'usp000fu3q', 'usp000g650', 'usp000gbc0',
       'usp000gs2d', 'usp000gv5p', 'usp000gz8j', 'usp000h05x',
       'usp000h1ys', 'usp000h237', 'usp000h2g9'

In [3]:
# Old station codes file
# AGENCY.DEPLOYMENT.STATION.--: LAT LON DEPTH BEGTIME ENDTIME

IR_stations = np.genfromtxt('/home/sdybing/neic-mlaapde/FDSN_IRstatcoord.dat', dtype = 'str', delimiter = ' ', usecols = (0,1))

IR_stations_test = IR_stations[29:34]

In [4]:
# Modern stations code file
# NET STAT LOC COMP: NORIENT EORIENT VERTORIENT: LAT LON DEPTH BEGTIME to ENDTIME

modern_stations = np.genfromtxt('/home/sdybing/neic-mlaapde/ALL_BHZstatcoord.dat', dtype = 'str', usecols = (0,1,2,3,7,8))

bad_format = ['AXBA1', 'AXCC1', 'AXEC2', 'HYSB1']

In [5]:
IR_stas_reshape = []

for idx in range(len(IR_stations)):
    IR_sta = IR_stations[idx][0].split(':')[0].split('.')[2]
    IR_stas_reshape.append(IR_sta)

IR_stasonly = np.array(IR_stas_reshape)

In [6]:
# Tohoku

# event_id = 'official20110311054624120_30'

# event_savedir = mseed_savedir + event_id + '/'
# if os.path.isdir(event_savedir):
#     pass
# else: # deletes directory to start over: shutil.rmtree(save_dir)  
#     os.makedirs(event_savedir)

# detail = get_event_by_id(event_id, includesuperseded = True)
# print(detail)

# df = get_phase_dataframe(detail)
# phase_data = df.to_numpy()

In [7]:
# Download data from IRIS

event_ids = new_catalog[:,8]

for event_id in event_ids:
    
    print('----------------------------------------------------------------------')
    event_savedir = mseed_savedir + event_id + '/'
    if os.path.isdir(event_savedir):
        pass
    else: # deletes directory to start over: shutil.rmtree(save_dir)  
        os.makedirs(event_savedir)

    # Get the phase picks from LibComCat
    detail = get_event_by_id(event_id, includesuperseded = True)
    print(detail)
    df = get_phase_dataframe(detail)
    phase_data = df.to_numpy()
    
    # Figure out the new station identifier and download and save data when possible
    mult_locs = []
    counter = 0

    for idx in range(len(phase_data)):
        print('--------------------------------')
        counter += 1
        print('Phase pick ' + str(counter) + '/' + str(len(phase_data)))
        row = phase_data[idx]
        net = row[0][0:2]
        sta = row[0][3:-6]
        phase = row[3]
        arrtime = UTCDateTime(row[4])

    #     print(str(net) + ', ' + str(sta) + ', ' + str(phase) + ', ' + str(arrtime))

        p = np.where(IR_stasonly == sta)[0]
    #     print(p)

        if len(p) == 0: # if the station associated with the phase pick isn't in the IR list
            IR_sta = sta

        elif len(p) > 1: # if there are multiple entries for the same station in the IR list because of multiple locations
            IR_sta = IR_stations[p[0]][0].split(':')[0].split('.')[2]
            IR_lat = float(IR_stations[p[0]][0].split(':')[1])
            IR_lon = float(IR_stations[p[0]][1])
            mult_locs.append(str(IR_sta) + '_num_locs:' + str(len(p)))

        else: # if len(p) = 1 (or negative numbers but that wouldn't happen)
            IR_sta = IR_stations[p[0]][0].split(':')[0].split('.')[2]
            IR_lat = float(IR_stations[p[0]][0].split(':')[1])
            IR_lon = float(IR_stations[p[0]][1])

    #     print('IR station ' + IR_sta + ': ' + str(IR_lat) + ', ' + str(IR_lon))

        try: # see if the IR station is in the modern_stations list
            w = np.where(modern_stations[:,1] == IR_sta)[0][0]
    #         print(w)
    #         print('IR sta: ' + IR_sta)
    #         print('Modern sta: ' + str(modern_stations[w,1]))
            status = True
            success = False

        except:
            print('No IR station ' + IR_sta + ' in modern BHZ list')
            print('Trying to access data anyway...')
            net = '*'
            sta = IR_sta
            loc = '*'
            a = 0
            b = 0

            try:
                E = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHE', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
                net = E[0].stats.network
                # E.plot()
                sta_savedir = event_savedir + net + '.' + sta + '.--/'
                if os.path.isdir(sta_savedir):
                    pass
                else: # deletes directory to start over: shutil.rmtree(save_dir)  
                    os.makedirs(sta_savedir)
                E.write(sta_savedir + 'BHE.mseed', format='MSEED')
                print('Saved BHE')
                a += 1
            except:
                print('No BHE')
                b += 1

            try:
                N = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHN', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
                net = N[0].stats.network
                # N.plot()
                sta_savedir = event_savedir + net + '.' + sta + '.--/'
                if os.path.isdir(sta_savedir):
                    pass
                else: # deletes directory to start over: shutil.rmtree(save_dir)  
                    os.makedirs(sta_savedir)
                N.write(sta_savedir + 'BHN.mseed', format='MSEED')
                print('Saved BHN')
                a += 1
            except:
                print('No BHN')
                b += 1

            try:
                Z = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHZ', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
                net = Z[0].stats.network
                # Z.plot()
                sta_savedir = event_savedir + net + '.' + sta + '.--/'
                if os.path.isdir(sta_savedir):
                    pass
                else: # deletes directory to start over: shutil.rmtree(save_dir)  
                    os.makedirs(sta_savedir)
                Z.write(sta_savedir + 'BHZ.mseed', format='MSEED')
                print('Saved BHZ')
                a += 1
            except:
                print('No BHZ')
                b += 1

            if b == 3:
                status = False

            if a > 0:
                success = True

        if status == False:
            print('Skipping - no data available')
            continue # skips the rest of this loop if there's no data available

        if success == True:
            continue # skips the rest of the loop if the data was already downloaded

        modern_net = modern_stations[w,0]
        modern_sta = modern_stations[w,1]
        modern_loc = modern_stations[w,2]
        modern_stacode = modern_net + '.' + modern_sta + '.' + modern_loc

        modern_lat = float(modern_stations[w,4])

        if modern_stations[w][1] in bad_format:
            modern_lon = float(modern_stations[w,5][0:9])
        else:
            modern_lon = float(modern_stations[w,5])

    #     print('Modern station ' + str(modern_stacode) + ': ' + str(modern_lat) + ', ' + str(modern_lon))

        if len(p) == 0: # from above - there will be no IR_lat or IR_lon
            j = False
            k = False

        else:
            j = np.isclose(IR_lat, modern_lat, atol = 0.0005)
            k = np.isclose(IR_lon, modern_lon, atol = 0.0005)

        if j == True and k == True: # Station name and location matches
    #         print('IR station ' + IR_sta + ': (' + str(IR_lat) + ', ' + str(IR_lon) + ')')
    #         print('Modern station ' + modern_stacode + ': (' + str(modern_lat) + ', ' + str(modern_lon) + ')')
            net = modern_net
            sta = modern_sta
            loc = modern_loc

        elif j == False and k == False: # Station name matches, but not location (like if it was missing from the IR file)
    #         print('IR station ' + IR_sta + '; not in IR file')
    #         print('Modern station ' + modern_stacode + ': ' + str(modern_lat) + ', ' + str(modern_lon))
            net = modern_net
            sta = modern_sta
            loc = modern_loc

        else:
            print('Locations do not match')
            status = False

        if status == False:
            print('Skipping')
            continue # skips the rest of this loop if the station locations don't match

        print(net + '.' + sta + '.' + loc)

        print('Getting data...')

        try:
            E = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHE', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
            # E.plot()
            sta_savedir = event_savedir + net + '.' + sta + '.' + loc + '/'
            if os.path.isdir(sta_savedir):
                pass
            else: # deletes directory to start over: shutil.rmtree(save_dir)  
                os.makedirs(sta_savedir)
            E.write(sta_savedir + 'BHE.mseed', format='MSEED')
            print('Saved BHE')
        except:
            print('No BHE')

        try:
            N = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHN', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
            # N.plot()
            sta_savedir = event_savedir + net + '.' + sta + '.' + loc + '/'
            if os.path.isdir(sta_savedir):
                pass
            else: # deletes directory to start over: shutil.rmtree(save_dir)  
                os.makedirs(sta_savedir)
            N.write(sta_savedir + 'BHN.mseed', format='MSEED')
            print('Saved BHN')
        except:
            print('No BHN')

        try:
            Z = client.get_waveforms(network = net, station = sta, location = loc, channel = 'BHZ', starttime = arrtime - pre_post_sec, endtime = arrtime + pre_post_sec)
            # Z.plot()
            sta_savedir = event_savedir + net + '.' + sta + '.' + loc + '/'
            if os.path.isdir(sta_savedir):
                pass
            else: # deletes directory to start over: shutil.rmtree(save_dir)  
                os.makedirs(sta_savedir)
            Z.write(sta_savedir + 'BHZ.mseed', format='MSEED')
            print('Saved BHZ')
        except:
            print('No BHZ')

    with open(event_savedir + 'mult_locs.txt', 'w') as f:
        for line in mult_locs:
            f.write(f"{line}\n")

----------------------------------------------------------------------
usp0009qb4 2000-03-28 11:00:22.510000 (22.338,143.730) 126.5 km M7.6
--------------------------------
Phase pick 1/633
IU.GUMO.00
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 2/633
JP.JOW.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 3/633
No IR station MAT in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 4/633
No IR station MJAR in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 5/633
IU.MAJO.00
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 6/633
JP.JNU.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 7/633
No IR station KSAR in modern BHZ list
Trying to access data any

No BHE
No BHN
No BHZ
--------------------------------
Phase pick 58/633
No IR station KNA in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 59/633
IU.YAK.00
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 60/633
No IR station SEY in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 61/633
No IR station BOD in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 62/633
IA.DNP.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 63/633
IU.CHTO.00
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 64/633
No IR station CMAR in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no 

No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 122/633
No IR station PMS in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 123/633
AU.TOO.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 124/633
AT.PMR.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 125/633
AK.MCK.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 126/633
KN.AML.--
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 127/633
AT.MID.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 128/633
IU.COLA.00
Getting data...
No BHE
No BHN
Saved BHZ
--------------------------------
Phase pick 129/633
No IR station HYB in modern BHZ list
Trying to access data anyway...
Saved BHE
Saved BHN
Saved BHZ
---------------------

No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 176/633
IU.COR.00
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 177/633
No IR station PECR in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 178/633
No IR station SHW in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 179/633
No IR station APA in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 180/633
No IR station LON in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 181/633
No IR station FMW in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no d

No BHE
No BHN
No BHZ
--------------------------------
Phase pick 233/633
US.HLID.00
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 234/633
No IR station ELMT in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 235/633
No IR station BUT in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 236/633
CI.ISA.--
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 237/633
No IR station HBMT in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 238/633
No IR station LRM in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 239/633
No IR station HRY in modern BHZ l

...

Saved BHZ
--------------------------------
Phase pick 584/861
IU.RAR.00
Getting data...
No BHE
No BHN
Saved BHZ
--------------------------------
Phase pick 585/861
GE.KBS.00
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 586/861
NO.SPA0.00
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 587/861
II.SHEL.00
Getting data...
No BHE
No BHN
Saved BHZ
--------------------------------
Phase pick 588/861
G.SSB.00
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 589/861
GE.WLF.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 590/861
IU.ADK.00
Getting data...
No BHE
No BHN
Saved BHZ
--------------------------------
Phase pick 591/861
IM.NC204.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 592/861
IM.NC200.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 593/861
IM.NBO03.--
Get

No BHE
No BHN
No BHZ
--------------------------------
Phase pick 662/861
IM.FIA1.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 663/861
No IR station FINES in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 664/861
PL.OJC.--
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 665/861
No IR station VAE in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 666/861
CI.BLY.--
Getting data...
No BHE
No BHN
No BHZ
--------------------------------
Phase pick 667/861
Locations do not match
Skipping
--------------------------------
Phase pick 668/861
MN.DIVS.--
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 669/861
No IR station TTG in modern BHZ list
Trying to access data anyway...
No BHE
No BHN


No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 717/861
No IR station PTL in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 718/861
No IR station AXAR in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 719/861
No IR station RDO in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 720/861
No IR station SMTH in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 721/861
No IR station ANKY in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 722/861
No IR station ALN in modern BHZ list
Try

No BHN
No BHZ
--------------------------------
Phase pick 774/861
No IR station KSAR in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 775/861
IU.INCN.00
Getting data...
No BHE
No BHN
Saved BHZ
--------------------------------
Phase pick 776/861
KR.NRN.--
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 777/861
No IR station MMAI in modern BHZ list
Trying to access data anyway...
No BHE
No BHN
No BHZ
Skipping - no data available
--------------------------------
Phase pick 778/861
Locations do not match
Skipping
--------------------------------
Phase pick 779/861
G.ATD.00
Getting data...
Saved BHE
Saved BHN
Saved BHZ
--------------------------------
Phase pick 780/861
IU.GUMO.00
Getting data...
No BHE
No BHN
Saved BHZ
--------------------------------
Phase pick 781/861
II.UOSS.00
Getting data...
No BHE
No BHN
Saved BHZ
--------------------------------
P

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

